## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [2]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [3]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.003813645918853581
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.15939615399111062
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.008109500049613416
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.23655258293729275


## Different MatMul Algorithms

In [57]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [58]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "tiled_register")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for tiled_register
loading cuda graph...
test run:
 [[  69.979675    -61.36071     122.15927    ...   28.837662
   -11.268646    -66.407295  ]
 [ 135.25233     108.83457      75.879814   ...  -32.02587
    -0.30557978   19.618298  ]
 [ -50.029224    -39.911236    -19.260012   ... -147.11209
    35.353226     11.227263  ]
 ...
 [  44.032215   -130.31781      -0.770494   ... -111.424736
    19.343025    -93.567795  ]
 [  68.003075   -120.62097      77.519165   ...   30.3433
    49.167793      9.773725  ]
 [  58.524734    -23.34882      21.666552   ...  -51.97913
  -104.595894     48.671562  ]]

reference:
 [[  69.979645   -61.360634   122.159065  ...   28.837719   -11.26861
   -66.40727  ]
 [ 135.25218    108.83465     75.87988   ...  -32.025932    -0.3057022
    19.618248 ]
 [ -50.02915    -39.911232   -19.260105  ... -147.1125      35.353195
    11.227253 ]
 ...
 [  44.032185  -130.31773     -0.7704153 ... -111.42463     19.34327
   -93.56785  ]
 [  68.003105  -120.

In [37]:
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The tiled matmul process looks like this:
![](tile.png)

The *tiled_register* version is further compute results in "subtiles":
![](subtile.png)

Run a complete benchmark for different algorithms:

In [ ]:
import time
for algo in ["naive", "coalescing", "tiled", "tiled_register"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.013117961399257184, 'mojo': 0.5179697485989891}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013877098605735227, 'mojo': 0.13656374839483762}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.013109807996079324, 'mojo': 0.11095175779773853}
building cuda graph for tiled_register
loading cuda graph...


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul